In [3]:
import math
import struct
angles_degrees = [45.0,        # 0.785398
                  26.56505118, # 0.46364760905065
                  14.03624347, # 0.24497866316245
                  7.125016349, # 0.1243549945481669
                  3.576334375, # 0.062418809995849
                  1.789910608, # 0.031239833425896
                  0.89517371,  # 0.01562372861675
                  0.447614171, # 0.0078123410625155
                  0.2239105,   # 0.003907975454777
                  0.111905677, # 0.0019531225153184
                  0.055952892, # 0.00097656219141037
                  0.027976453, # 0.00048828121787823
                  0.013988227, # 0.00024414061766576
                  0.006994114, # 0.000122070317559528
                  0.003497057, # 6.103515877976e-5
                  0.001748528  # 3.051757066324e-5
                  ]



def scale_input(angle):
    # Ensure angle is within the range [-pi, pi)
    angle = angle % (2 * math.pi)
    # Ensure angle is positive
    if angle < 0:
        angle += 2 * math.pi
    
    # Determine the number of pi/2 rotations needed to bring the angle to the first quadrant
    num_rotations = math.floor(angle / (0.5 * math.pi))
    
    # Compute the scaled angle in the first quadrant
    scaled_angle = angle - num_rotations * (0.5 * math.pi)
    
    # Determine the sign based on the number of rotations
    sign = 1 if num_rotations % 2 == 0 else - 1
    
    return scaled_angle, sign

def CORDIC_TEST(angle_deg):
      
    #a test of cordic
    print(f"actual Value: {math.cos(math.radians(angle_deg))}\n")
    
    angle = math.radians(angle_deg)
    angle, sign = scale_input(angle)
    angles_radians = [math.radians(i) for i in angles_degrees]
    
    z = 0.0                     # angle
    x = 1.0 / 1.646760258121    # x
    y = 0.0                     # y
    #fix 7 iterations
    for i in range(8):
        print(f"z: {math.degrees(z)}, x: {x}, y: {y}")
        if (z < angle):
            print("too small")
            z = z + angles_radians[i]
            x_plus = x - (y * 2 ** (-i))
            y_plus = y + (x * 2 ** (-i))
            
        elif (z > angle):
            print("too big")
            z = z - angles_radians[i]
            x_plus = x + (y * 2 ** (-i))
            y_plus = y - (x * 2 ** (-i))
            
        else:
            print(f"Exact, Predicted Value: z = {math.degrees(z)} with cos(x) = {x}")
            return
        

        x = x_plus
        y = y_plus
        
    print(f"\nPredicted Value: z = {math.degrees(z)} with cos(x) = {x}, with actual value of predicted: {math.cos(z)}")
    print(f"The result will have a sign of: {sign}")
    
    

CORDIC_TEST(42.0)





actual Value: 0.7071067811865476

z: 0.0, x: 0.6072529350089055, y: 0.0
too small
z: 45.0, x: 0.6072529350089055, y: 0.6072529350089055
Exact, Predicted Value: z = 45.0 with cos(x) = 0.6072529350089055


# Testing Input Scaling

In [61]:

TEST = 330.0
out, sign = scale_input(math.radians(TEST))
print(f"Scaling {(TEST)} to be equal to {math.degrees(out)} with sign {sign}")

Scaling 330.0 to be equal to 59.999999999999986 with sign -1


# Generate Files

In [81]:
# Number of sampling intervals
FLOAT_BITS = 32
UPPER_BITS = 16
LOWER_BITS = 16


def float_to_fixed_point(float_num, upper_bits, fractional_bits):
    print(float_num)
    integer_part = int(float_num)
    fractional_part = math.modf(float_num)[0]  # modf returns a tuple (fractional, integer)
    fractional_part *= 10**10
    print(f"int: {integer_part}, frac; {fractional_part}")

    #return binary_result



def write_to_mem_file(filename, values):
    with open(filename, 'w') as f:
        for i, val in enumerate(values):
            binary_val = float_to_fixed_point(val, UPPER_BITS, LOWER_BITS)
            #f.write(f'4\'b{bin(i)[2:].zfill(4)} <= 32\'b{binary_val};\n')
            f.write(f'CORDIC_shifts[4\'b{bin(i)[2:].zfill(4)}] <= 32\'b{binary_val};\n')






#make_cordic_iters(FLOAT_BITS, UPPER_BITS, LOWER_BITS)
values = [math.radians(i) for i in angles_degrees]
write_to_mem_file("CORDIC_shifts_radians.mem", values)



0.7853981633974483
int: 0, frac; 7853981633.974483
0.46364760905180485
int: 0, frac; 4636476090.518048
0.24497866316305394
int: 0, frac; 2449786631.6305394
0.12435499454847539
int: 0, frac; 1243549945.4847538
0.062418809996003584
int: 0, frac; 624188099.9600358
0.031239833425973557
int: 0, frac; 312398334.2597356
0.01562372861679289
int: 0, frac; 156237286.1679289
0.0078123410625349195
int: 0, frac; 78123410.6253492
0.003907975454786763
int: 0, frac; 39079754.54786763
0.0019531225153232904
int: 0, frac; 19531225.153232902
0.0009765621914127951
int: 0, frac; 9765621.914127951
0.00048828121787944514
int: 0, frac; 4882812.178794451
0.00024414061766636885
int: 0, frac; 2441406.1766636884
0.00012207031755983067
int: 0, frac; 1220703.1755983068
6.103515877991534e-05
int: 0, frac; 610351.5877991534
3.051757066331141e-05
int: 0, frac; 305175.70663311414


# ERRORS

 mean error e is in the range [-0.5x10-6 , 0.5x10-6] withconfidence level of 95%, assuming an input x is under single precision format and follows a uniform distribution in the range [-1, 1] (i.e. the possible range of input given our problem). 